# Business Analyst Workflow - ED-AI Metadata Bridge

**Use Case**: Sales analyst comparing company performance with international trends

## Workflow with Local AI Calculator

```text
1. Analyst writes natural language query
   ↓
2. 🤖 Local AI refines prompt (adds context, structure)
   ↓
3. System converts query → HLLSet fingerprint
   ↓
4. Query knowledge base (documents as HLLSets)
   ↓
5. Retrieve similar documents via disambiguation
   ↓
6. Send documents → 🌐 External AI → Summary
   ↓
7. Analyst edits summary
   ↓
8. 🤖 Local AI converts NL question → SQL query
   ↓
9. Query database (via metadata lattice) → Retrieve sales data
   ↓
10. 🤖 Local AI summarizes database results
   ↓
11. Analyst adds comments/concerns
   ↓
12. 🤖 Local AI formats context for external AI
   ↓
13. Send to 🌐 External AI → Final strategic analysis
   ↓
14. Present results to analyst
```

**Legend:**
- 🤖 Local AI = Fast, private, on-premises (Ollama/GPT4All)
- 🌐 External AI = Deep analysis (OpenAI/Anthropic/Claude)

## Architecture

- **Local AI Calculator**: Prompt refinement, NL→SQL, data summarization
- **Data Perceptron**: Documents ingested as HLLSets
- **Metadata Perceptron**: Database schema as lattice
- **Entanglement**: Query HLLSet ↔ Document HLLSets
- **Unified Storage**: All artifacts content-addressed
- **Hybrid AI**: Local for routine tasks, external for strategic analysis

In [2]:
# Imports
from core.extensions.unified_storage import (
    UnifiedStorageExtension, PerceptronConfig, LatticeNode, LatticeEdge
)
from core.hllset import HLLSet, compute_sha1
from collections import defaultdict
import json
from datetime import datetime

## Step 1: Initialize System

Create unified storage with data and metadata perceptrons.

In [3]:
# Initialize unified storage
storage = UnifiedStorageExtension(":memory:")

# Register data perceptron (for documents)
data_config = PerceptronConfig(
    perceptron_id="documents_perceptron",
    perceptron_type="data",
    hash_function="sha1",
    hash_seed=42,
    config_dict={"p_bits": 14, "description": "Document knowledge base"}
)
storage.register_perceptron(data_config, "Documents knowledge base perceptron")

# Register metadata perceptron (for database)
metadata_config = PerceptronConfig(
    perceptron_id="sales_db_perceptron",
    perceptron_type="metadata",
    hash_function="sha1",
    hash_seed=99,
    config_dict={"p_bits": 12, "description": "Sales database metadata"}
)
storage.register_perceptron(metadata_config, "Sales database metadata perceptron")

print("✓ System initialized")
print(f"  Perceptrons: {len(storage.list_perceptrons())}")

✓ System initialized
  Perceptrons: 2


## Step 1.5: Initialize Local AI Calculator

Create a local AI assistant for routine tasks:
- Prompt engineering and refinement
- Natural language → SQL conversion
- Data formatting and quick summaries
- No external API calls (fast, private, cost-effective)

**Note**: This uses a mock implementation. In production, use:
- **Ollama** (llama3, mistral, codellama)
- **GPT4All** (local inference)
- **llama.cpp** (optimized local models)

In [4]:
class LocalAICalculator:
    """
    Local AI assistant for routine analyst tasks.
    
    In production, this would use:
    - Ollama with llama3/mistral/codellama
    - GPT4All for local inference
    - llama.cpp for optimized models
    
    Benefits:
    - Fast response (no network latency)
    - Private (data stays local)
    - Cost-effective (no API fees)
    - Offline capable
    """
    
    def __init__(self, model_name="mock-local-llm"):
        self.model_name = model_name
        print(f"🤖 Initialized Local AI Calculator: {model_name}")
    
    def refine_prompt(self, user_query: str, context: dict = None) -> str:
        """
        Help analyst refine their natural language query.
        Adds structure, clarity, and missing context.
        """
        # In production: Call local Ollama/GPT4All
        # For demo: Rule-based enhancement
        
        refined = user_query.strip()
        
        # Add time context if missing
        if "time" not in refined.lower() and "period" not in refined.lower():
            refined += "\n[Auto-added] Time period: Recent quarter (Q4 2025)"
        
        # Add specificity suggestions
        if "compare" in refined.lower() and "metrics" not in refined.lower():
            refined += "\n[Suggested metrics: growth rate, market share, revenue, volume]"
        
        # Add output format guidance
        refined += "\n[Preferred output: Executive summary with actionable insights]"
        
        return refined
    
    def nl_to_sql(self, question: str, schema_info: list) -> str:
        """
        Convert natural language question to SQL query.
        Uses database schema metadata from metadata lattice.
        """
        # In production: Use codellama or specialized SQL model
        # For demo: Template-based generation
        
        question_lower = question.lower()
        
        # Extract table names from schema
        tables = [t['table_name'] for t in schema_info]
        
        # Pattern matching for common queries
        if "sales" in question_lower and "region" in question_lower:
            if "product" in question_lower or "category" in question_lower:
                return f"""
SELECT 
    p.category,
    r.name as region,
    SUM(s.quantity) as units_sold,
    SUM(s.revenue) as total_revenue,
    AVG(s.revenue / s.quantity) as avg_price
FROM sales s
JOIN products p ON s.product_id = p.product_id
JOIN regions r ON s.region = r.region_id
WHERE s.sale_date BETWEEN '2025-10-01' AND '2025-12-31'
GROUP BY p.category, r.name
ORDER BY total_revenue DESC
"""
        
        if "top" in question_lower and "product" in question_lower:
            return f"""
SELECT 
    p.name as product_name,
    p.category,
    SUM(s.quantity) as total_units,
    SUM(s.revenue) as total_revenue
FROM sales s
JOIN products p ON s.product_id = p.product_id
WHERE s.sale_date >= '2025-10-01'
GROUP BY p.product_id, p.name, p.category
ORDER BY total_revenue DESC
LIMIT 10
"""
        
        if "trend" in question_lower or "growth" in question_lower:
            return f"""
SELECT 
    DATE_TRUNC('month', s.sale_date) as month,
    p.category,
    SUM(s.revenue) as monthly_revenue,
    SUM(s.quantity) as monthly_units
FROM sales s
JOIN products p ON s.product_id = p.product_id
WHERE s.sale_date >= '2025-01-01'
GROUP BY DATE_TRUNC('month', s.sale_date), p.category
ORDER BY month, p.category
"""
        
        # Generic fallback
        return f"""
SELECT * FROM sales 
WHERE sale_date >= '2025-10-01'
LIMIT 100
"""
    
    def summarize_data(self, data: list, summary_type: str = "quick") -> str:
        """
        Create quick summaries of database results.
        Useful for presenting data to analyst before external AI.
        """
        if not data:
            return "No data returned"
        
        if summary_type == "quick":
            row_count = len(data)
            
            # Find numeric columns
            numeric_cols = []
            if data:
                for key, value in data[0].items():
                    if isinstance(value, (int, float)):
                        numeric_cols.append(key)
            
            summary = f"📊 Quick Summary:\n"
            summary += f"  • Rows: {row_count}\n"
            
            # Calculate basic stats for numeric columns
            if numeric_cols:
                for col in numeric_cols[:3]:  # Top 3 numeric columns
                    values = [row[col] for row in data if col in row]
                    if values:
                        total = sum(values)
                        avg = total / len(values)
                        summary += f"  • {col}: Total={total:,.0f}, Avg={avg:,.1f}\n"
            
            return summary
        
        elif summary_type == "detailed":
            # More comprehensive analysis
            summary = f"📈 Detailed Analysis:\n"
            summary += f"  • Total records: {len(data)}\n"
            
            if data:
                summary += f"  • Columns: {', '.join(data[0].keys())}\n"
                
                # Group by logic if applicable
                if 'category' in data[0]:
                    categories = set(row['category'] for row in data)
                    summary += f"  • Categories: {len(categories)} ({', '.join(list(categories)[:5])})\n"
                
                if 'region' in data[0]:
                    regions = set(row['region'] for row in data)
                    summary += f"  • Regions: {len(regions)} ({', '.join(regions)})\n"
            
            return summary
        
        return "Summary type not recognized"
    
    def format_for_external_ai(self, context: dict) -> str:
        """
        Format data and context for external AI consumption.
        Structures information to minimize token usage.
        """
        formatted = "CONTEXT FOR EXTERNAL AI:\n"
        formatted += "=" * 60 + "\n\n"
        
        if 'query' in context:
            formatted += f"ANALYST QUERY:\n{context['query']}\n\n"
        
        if 'documents' in context:
            formatted += f"RELEVANT DOCUMENTS ({len(context['documents'])}):\n"
            for doc in context['documents'][:3]:  # Top 3
                formatted += f"  • {doc.get('doc_id', 'Unknown')}: "
                formatted += f"{doc.get('content', '')[:100]}...\n"
            formatted += "\n"
        
        if 'database_results' in context:
            data = context['database_results']
            formatted += f"DATABASE RESULTS:\n"
            formatted += self.summarize_data(data, "detailed")
            formatted += f"\n  [Full data: {len(data)} rows]\n\n"
        
        if 'analyst_notes' in context:
            formatted += f"ANALYST NOTES:\n{context['analyst_notes']}\n\n"
        
        formatted += "=" * 60
        
        return formatted

# Initialize local AI calculator
local_ai = LocalAICalculator(model_name="mock-local-llm-v1")

print("✓ Local AI Calculator ready")
print("  Capabilities:")
print("    • Prompt refinement")
print("    • Natural language → SQL")
print("    • Data summarization")
print("    • Context formatting")

🤖 Initialized Local AI Calculator: mock-local-llm-v1
✓ Local AI Calculator ready
  Capabilities:
    • Prompt refinement
    • Natural language → SQL
    • Data summarization
    • Context formatting


## Step 2: Ingest Knowledge Base (Documents)

Simulate ingesting various documents into the system:
- Market research reports
- Internal sales reports
- Industry trend analyses
- Competitor analyses

In [5]:
# Sample documents (in real system, these would be much larger)
documents = {
    "doc_001_market_trends_2025": """
        Global electronics market trends Q4 2025. Smartphone sales increased 15% YoY.
        Laptop sales grew 8% driven by remote work. Tablet market declined 3%.
        Consumer electronics revenue reached $1.2T globally. Asia-Pacific leading growth.
    """,
    "doc_002_competitor_analysis": """
        Competitor analysis: TechCorp increased market share to 22% in smartphones.
        ElectroGiant maintained 18% share in laptops. NewTech disrupting tablet market
        with innovative products. Price competition intensifying across all categories.
    """,
    "doc_003_internal_sales_q4": """
        Internal sales report Q4 2025. Smartphone unit sales: 2.5M units (+12% QoQ).
        Laptop sales: 800K units (+5% QoQ). Tablet sales: 400K units (-2% QoQ).
        Total revenue: $450M. Profit margin: 18%. Customer satisfaction: 87%.
    """,
    "doc_004_industry_forecast": """
        Industry forecast 2026: Smartphone market expected to grow 10-12% annually.
        AI-powered devices driving innovation. 5G adoption accelerating replacement cycles.
        Sustainability concerns influencing purchasing decisions. Supply chain stabilizing.
    """,
    "doc_005_customer_feedback": """
        Customer feedback analysis: High satisfaction with smartphone quality and performance.
        Laptop battery life concerns mentioned frequently. Tablet users want larger screens.
        Price sensitivity increasing in budget segment. Premium segment growing steadily.
    """
}

# Ingest documents as HLLSets
document_hllsets = {}
for doc_id, content in documents.items():
    # Tokenize and create HLLSet
    tokens = content.lower().split()
    hllset = HLLSet.from_batch(tokens, p_bits=14)
    
    # Store HLLSet
    storage.store_hllset(hllset)
    document_hllsets[doc_id] = {
        'hllset': hllset,
        'content': content.strip(),
        'hash': hllset.name
    }

print(f"✓ Ingested {len(document_hllsets)} documents into knowledge base")
for doc_id in document_hllsets.keys():
    print(f"  • {doc_id}")

✓ Ingested 5 documents into knowledge base
  • doc_001_market_trends_2025
  • doc_002_competitor_analysis
  • doc_003_internal_sales_q4
  • doc_004_industry_forecast
  • doc_005_customer_feedback


## Step 3: Setup Database Schema (Metadata Lattice)

Store sales database schema as metadata lattice.

In [6]:
# Create metadata lattice for sales database
db_lattice_id = storage.create_lattice(
    perceptron_id="sales_db_perceptron",
    lattice_type="metadata",
    dimension=10,
    config={"database": "sales_db", "schema": "public"}
)

# Define sales database schema
sales_tables = [
    {
        "name": "products",
        "columns": ["product_id", "name", "category", "price", "launch_date"],
        "rows": 150,
        "sample_data": "Smartphones, Laptops, Tablets"
    },
    {
        "name": "sales",
        "columns": ["sale_id", "product_id", "quantity", "revenue", "sale_date", "region"],
        "rows": 1250000,
        "sample_data": "Q4 2025 sales transactions"
    },
    {
        "name": "regions",
        "columns": ["region_id", "name", "country", "market_size"],
        "rows": 50,
        "sample_data": "North America, Europe, Asia-Pacific"
    },
    {
        "name": "customers",
        "columns": ["customer_id", "name", "segment", "region_id"],
        "rows": 500000,
        "sample_data": "Consumer, Enterprise, Government"
    }
]

# Store tables as nodes
table_nodes = {}
for idx, table in enumerate(sales_tables):
    node_id = compute_sha1(f"sales_table:{table['name']}")
    
    node = LatticeNode(
        node_id=node_id,
        node_index=idx,
        node_type="meta_table",
        content_hash=compute_sha1(table['name']),
        cardinality=float(table['rows']),
        properties={
            "table_name": table['name'],
            "columns": table['columns'],
            "row_count": table['rows'],
            "sample_data": table['sample_data']
        }
    )
    
    storage.store_lattice_node(db_lattice_id, node)
    table_nodes[table['name']] = node_id

# Add foreign key relationships
fk_relationships = [
    ("sales", "products", "product_id"),
    ("sales", "regions", "region"),
    ("customers", "regions", "region_id"),
]

for src_table, tgt_table, fk_col in fk_relationships:
    edge_id = compute_sha1(f"fk:{src_table}:{tgt_table}:{fk_col}")
    
    edge = LatticeEdge(
        edge_id=edge_id,
        source_node=table_nodes[src_table],
        target_node=table_nodes[tgt_table],
        edge_type="meta_fk",
        weight=1.0,
        properties={"fk_column": fk_col}
    )
    
    storage.store_lattice_edge(db_lattice_id, edge)

print(f"✓ Database schema stored in metadata lattice")
print(f"  Tables: {len(sales_tables)}")
print(f"  Foreign keys: {len(fk_relationships)}")

✓ Database schema stored in metadata lattice
  Tables: 4
  Foreign keys: 3


## Step 4: Analyst Query

Business analyst writes natural language query.

In [7]:
# Analyst's natural language query (original)
analyst_query_raw = """
    Compare our Q4 2025 smartphone and laptop sales performance against 
    international market trends. Focus on growth rates, market share, and 
    competitive positioning. Include customer feedback and industry forecasts 
    for 2026. Goal: Identify opportunities for growth in Asia-Pacific region.
"""

print("📝 Original Analyst Query:")
print(analyst_query_raw)

# Use Local AI to refine the prompt
analyst_query = local_ai.refine_prompt(analyst_query_raw)

print("\n✨ Local AI Refined Query:")
print(analyst_query)
print("\n" + "="*70)

📝 Original Analyst Query:

    Compare our Q4 2025 smartphone and laptop sales performance against 
    international market trends. Focus on growth rates, market share, and 
    competitive positioning. Include customer feedback and industry forecasts 
    for 2026. Goal: Identify opportunities for growth in Asia-Pacific region.


✨ Local AI Refined Query:
Compare our Q4 2025 smartphone and laptop sales performance against 
    international market trends. Focus on growth rates, market share, and 
    competitive positioning. Include customer feedback and industry forecasts 
    for 2026. Goal: Identify opportunities for growth in Asia-Pacific region.
[Auto-added] Time period: Recent quarter (Q4 2025)
[Suggested metrics: growth rate, market share, revenue, volume]
[Preferred output: Executive summary with actionable insights]



## Step 5: Convert Query to HLLSet

System converts natural language query to HLLSet fingerprint.

In [8]:
# Tokenize query
query_tokens = analyst_query.lower().split()

# Create query HLLSet
query_hllset = HLLSet.from_batch(query_tokens, p_bits=14)

print(f"✓ Query converted to HLLSet")
print(f"  Query tokens: {len(query_tokens)}")
print(f"  Query HLLSet cardinality: {query_hllset.cardinality():.1f}")
print(f"  Query hash: {query_hllset.name[:20]}...")

✓ Query converted to HLLSet
  Query tokens: 60
  Query HLLSet cardinality: 50.0
  Query hash: 0b1f76835e894a8596c2...


## Step 6: Query Knowledge Base

Find relevant documents by computing similarity between query HLLSet and document HLLSets.

In [9]:
# Compute similarity scores
similarities = []
for doc_id, doc_data in document_hllsets.items():
    similarity = query_hllset.similarity(doc_data['hllset'])
    similarities.append((doc_id, similarity, doc_data))

# Sort by similarity (descending)
similarities.sort(key=lambda x: x[1], reverse=True)

print("🔍 Document Similarity Scores:")
print("-" * 70)
for doc_id, similarity, _ in similarities:
    print(f"  {doc_id}: {similarity:.3f}")

# Select top 3 most relevant documents
relevant_docs = similarities[:3]
print(f"\n✓ Selected top {len(relevant_docs)} relevant documents")

🔍 Document Similarity Scores:
----------------------------------------------------------------------
  doc_005_customer_feedback: 0.093
  doc_001_market_trends_2025: 0.083
  doc_003_internal_sales_q4: 0.071
  doc_004_industry_forecast: 0.053
  doc_002_competitor_analysis: 0.040

✓ Selected top 3 relevant documents


## Step 7: Reconstruct Documents & Send to AI

Retrieve document content and send to external AI for summarization.

**Note**: In production, this would call an actual AI API (OpenAI, Anthropic, etc.)

In [10]:
# Reconstruct document content
retrieved_content = []
for doc_id, similarity, doc_data in relevant_docs:
    retrieved_content.append({
        'doc_id': doc_id,
        'similarity': similarity,
        'content': doc_data['content']
    })

print("📄 Retrieved Documents:")
print("="*70)
for item in retrieved_content:
    print(f"\n[{item['doc_id']}] (similarity: {item['similarity']:.3f})")
    print(item['content'][:200] + "...")

# Simulate AI summarization (in production, call actual AI API)
def simulate_ai_summary(query, documents):
    """Simulate AI generating summary from documents."""
    return f"""
AI SUMMARY (Generated from {len(documents)} documents):

Q4 2025 Performance Analysis:
• Your smartphone sales grew 12% QoQ (2.5M units), slightly below the 15% 
  global market growth rate
• Laptop sales increased 5% QoQ (800K units), underperforming the 8% 
  global growth
• Total revenue: $450M with 18% profit margin

Market Position:
• Competitors like TechCorp (22% market share) and ElectroGiant (18%) 
  are gaining ground
• Your market share in smartphones and laptops needs strategic attention

Opportunities for Asia-Pacific Growth:
• Region leading global electronics growth
• Strong demand for 5G smartphones and AI-powered devices
• Customer feedback indicates high satisfaction with smartphone quality
• 2026 forecast: 10-12% annual growth in smartphone segment

Recommendations:
1. Increase investment in Asia-Pacific distribution and marketing
2. Address laptop battery life concerns mentioned in customer feedback
3. Leverage 5G adoption cycle for smartphone replacement demand
4. Focus on premium segment where growth is steady
"""

ai_summary = simulate_ai_summary(analyst_query, retrieved_content)

print("\n" + "="*70)
print("🤖 AI SUMMARY:")
print("="*70)
print(ai_summary)

📄 Retrieved Documents:

[doc_005_customer_feedback] (similarity: 0.093)
Customer feedback analysis: High satisfaction with smartphone quality and performance.
        Laptop battery life concerns mentioned frequently. Tablet users want larger screens.
        Price sensit...

[doc_001_market_trends_2025] (similarity: 0.083)
Global electronics market trends Q4 2025. Smartphone sales increased 15% YoY.
        Laptop sales grew 8% driven by remote work. Tablet market declined 3%.
        Consumer electronics revenue reache...

[doc_003_internal_sales_q4] (similarity: 0.071)
Internal sales report Q4 2025. Smartphone unit sales: 2.5M units (+12% QoQ).
        Laptop sales: 800K units (+5% QoQ). Tablet sales: 400K units (-2% QoQ).
        Total revenue: $450M. Profit margin...

🤖 AI SUMMARY:

AI SUMMARY (Generated from 3 documents):

Q4 2025 Performance Analysis:
• Your smartphone sales grew 12% QoQ (2.5M units), slightly below the 15% 
  global market growth rate
• Laptop sales increased 5

## Step 8: Analyst Reviews & Edits Summary

Analyst can review and edit the AI-generated summary.

In [ ]:
# Simulate analyst editing the summary
edited_summary = ai_summary + """

[ANALYST EDITS]:
• Need to verify actual sales data from database for Q4 2025
• Want to see detailed breakdown by product category and region
• Concerned about 3-point gap vs market growth rate in smartphones
• Question: What were our specific sales figures in Asia-Pacific?
"""

print("✓ Analyst reviewed and edited summary")
print("\nEdited version includes:")
print("  • Request for database verification")
print("  • Need for detailed regional breakdown")
print("  • Specific concerns about growth gap")

## Step 9: Query Database via Metadata Lattice

System uses metadata lattice to construct and execute database queries.

In [11]:
# Query metadata lattice to understand available data
meta_nodes = storage.get_lattice_nodes(db_lattice_id, node_type="meta_table")

print("📊 Available Database Tables:")
print("-" * 70)
schema_info = []
for node in meta_nodes:
    table = node['properties']['table_name']
    rows = node['properties']['row_count']
    cols = node['properties']['columns']
    print(f"\n• {table} ({rows:,} rows)")
    print(f"  Columns: {', '.join(cols)}")
    print(f"  Sample: {node['properties']['sample_data']}")
    
    schema_info.append({
        'table_name': table,
        'columns': cols,
        'row_count': rows,
        'sample_data': node['properties']['sample_data']
    })

# Use Local AI to convert natural language question to SQL
analyst_question = "Show me sales breakdown by product category and region for Q4 2025"

print("\n" + "="*70)
print("🤖 Local AI: Converting NL → SQL")
print("="*70)
print(f"\nQuestion: {analyst_question}")

generated_sql = local_ai.nl_to_sql(analyst_question, schema_info)

print(f"\nGenerated SQL:")
print(generated_sql)

# Simulate database query results
# In production, this would execute the generated SQL
simulated_db_results = {
    "query": generated_sql.strip(),
    "results": [
        {"category": "Smartphones", "region": "Asia-Pacific", "units_sold": 1200000, "revenue": 240000000},
        {"category": "Smartphones", "region": "North America", "units_sold": 800000, "revenue": 160000000},
        {"category": "Smartphones", "region": "Europe", "units_sold": 500000, "revenue": 100000000},
        {"category": "Laptops", "region": "North America", "units_sold": 350000, "revenue": 87500000},
        {"category": "Laptops", "region": "Asia-Pacific", "units_sold": 280000, "revenue": 70000000},
        {"category": "Laptops", "region": "Europe", "units_sold": 170000, "revenue": 42500000},
        {"category": "Tablets", "region": "North America", "units_sold": 200000, "revenue": 30000000},
        {"category": "Tablets", "region": "Asia-Pacific", "units_sold": 150000, "revenue": 22500000},
        {"category": "Tablets", "region": "Europe", "units_sold": 50000, "revenue": 7500000},
    ]
}

print("\n" + "="*70)
print("💾 DATABASE QUERY RESULTS:")
print("="*70)
print(f"\nSQL Query:\n{simulated_db_results['query']}")
print("\nResults:")
print("-" * 70)
print(f"{'Category':<15} {'Region':<20} {'Units Sold':>15} {'Revenue':>15}")
print("-" * 70)
for row in simulated_db_results['results']:
    print(f"{row['category']:<15} {row['region']:<20} {row['units_sold']:>15,} ${row['revenue']:>14,}")

# Calculate totals
total_units = sum(r['units_sold'] for r in simulated_db_results['results'])
total_revenue = sum(r['revenue'] for r in simulated_db_results['results'])
print("-" * 70)
print(f"{'TOTAL':<36} {total_units:>15,} ${total_revenue:>14,}")

# Calculate Asia-Pacific percentage
apac_revenue = sum(r['revenue'] for r in simulated_db_results['results'] if r['region'] == 'Asia-Pacific')
apac_percentage = (apac_revenue / total_revenue) * 100
print(f"\n✓ Asia-Pacific represents {apac_percentage:.1f}% of Q4 2025 revenue")

📊 Available Database Tables:
----------------------------------------------------------------------

• products (150 rows)
  Columns: product_id, name, category, price, launch_date
  Sample: Smartphones, Laptops, Tablets

• sales (1,250,000 rows)
  Columns: sale_id, product_id, quantity, revenue, sale_date, region
  Sample: Q4 2025 sales transactions

• regions (50 rows)
  Columns: region_id, name, country, market_size
  Sample: North America, Europe, Asia-Pacific

• customers (500,000 rows)
  Columns: customer_id, name, segment, region_id
  Sample: Consumer, Enterprise, Government

🤖 Local AI: Converting NL → SQL

Question: Show me sales breakdown by product category and region for Q4 2025

Generated SQL:

SELECT 
    p.category,
    r.name as region,
    SUM(s.quantity) as units_sold,
    SUM(s.revenue) as total_revenue,
    AVG(s.revenue / s.quantity) as avg_price
FROM sales s
JOIN products p ON s.product_id = p.product_id
JOIN regions r ON s.region = r.region_id
WHERE s.sale_date B

## Step 10: Analyst Adds Comments & Concerns

Analyst reviews database results and adds business context.

## Step 9.5: Local AI Data Summarization

Use local AI to quickly summarize database results before presenting to analyst.

In [12]:
# Use Local AI to summarize database results
print("\n🤖 Local AI: Quick Data Summary")
print("="*70)

quick_summary = local_ai.summarize_data(simulated_db_results['results'], "quick")
print(quick_summary)

print("\n🤖 Local AI: Detailed Analysis")
print("="*70)

detailed_summary = local_ai.summarize_data(simulated_db_results['results'], "detailed")
print(detailed_summary)

print("\n✓ Local AI provided instant insights without external API call")


🤖 Local AI: Quick Data Summary
📊 Quick Summary:
  • Rows: 9
  • units_sold: Total=3,700,000, Avg=411,111.1
  • revenue: Total=760,000,000, Avg=84,444,444.4


🤖 Local AI: Detailed Analysis
📈 Detailed Analysis:
  • Total records: 9
  • Columns: category, region, units_sold, revenue
  • Categories: 3 (Laptops, Tablets, Smartphones)
  • Regions: 3 (Europe, Asia-Pacific, North America)


✓ Local AI provided instant insights without external API call


In [14]:
analyst_comments = """
ANALYST COMMENTS & CONCERNS:

Key Findings from Database:
• Asia-Pacific is our strongest market (48% of smartphone revenue)
• Smartphone revenue: $500M total, with $240M from Asia-Pacific
• Laptop revenue: $200M total, strongest in North America
• Tablets underperforming: only $60M revenue

Concerns:
1. Despite strong Asia-Pacific performance, our 12% QoQ growth in smartphones
   is below the 15% global market growth rate
2. Laptop sales growth of 5% QoQ significantly trails 8% market growth
3. Market share appears to be declining relative to TechCorp (22%) and 
   ElectroGiant (18%)
4. Tablet segment declining (-2% QoQ) while market is only down 3%

Questions for AI Analysis:
1. What specific strategies should we implement to close the growth gap?
2. Should we increase Asia-Pacific investment given it's already our 
   strongest region?
3. How can we improve laptop competitiveness without cannibalizing 
   smartphone sales?
4. Is the tablet segment worth continued investment or should we reallocate
   resources?
5. What competitive advantages can we leverage based on customer feedback?
"""

print("📝 Analyst Comments:")
print("="*70)
print(analyst_comments)

📝 Analyst Comments:

ANALYST COMMENTS & CONCERNS:

Key Findings from Database:
• Asia-Pacific is our strongest market (48% of smartphone revenue)
• Smartphone revenue: $500M total, with $240M from Asia-Pacific
• Laptop revenue: $200M total, strongest in North America
• Tablets underperforming: only $60M revenue

Concerns:
1. Despite strong Asia-Pacific performance, our 12% QoQ growth in smartphones
   is below the 15% global market growth rate
2. Laptop sales growth of 5% QoQ significantly trails 8% market growth
3. Market share appears to be declining relative to TechCorp (22%) and 
   ElectroGiant (18%)
4. Tablet segment declining (-2% QoQ) while market is only down 3%

Questions for AI Analysis:
1. What specific strategies should we implement to close the growth gap?
2. Should we increase Asia-Pacific investment given it's already our 
   strongest region?
3. How can we improve laptop competitiveness without cannibalizing 
   smartphone sales?
4. Is the tablet segment worth continue

## Step 11: Send to AI for Final Analysis

System sends edited summary + database results + analyst comments to AI.

In [15]:
# Use Local AI to format context for external AI
# This optimizes token usage and structures data efficiently
print("🤖 Local AI: Formatting context for external AI")
print("="*70)

context_dict = {
    'query': analyst_query,
    'documents': retrieved_content,
    'database_results': simulated_db_results['results'],
    'analyst_notes': analyst_comments
}

final_ai_context = local_ai.format_for_external_ai(context_dict)

print("\nFormatted context (optimized for external AI):")
print(final_ai_context[:500] + "...\n")

print(f"✓ Context prepared: {len(final_ai_context)} characters")
print(f"  (Local AI reduced redundancy and optimized structure)\n")

# Simulate final AI analysis
def simulate_final_ai_analysis(context):
    """Simulate AI providing strategic recommendations."""
    return """
🤖 AI STRATEGIC ANALYSIS:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

EXECUTIVE SUMMARY:
Your company has strong positioning in Asia-Pacific but is losing market
share globally. The data reveals a strategic opportunity to leverage your
regional strength while addressing competitive gaps.

STRATEGIC RECOMMENDATIONS:

1. DOUBLE DOWN ON ASIA-PACIFIC (Priority: HIGH)
   • Already 48% of smartphone revenue with room for growth
   • Market forecast: 10-12% annual growth through 2026
   • Action: Increase marketing budget by 30% in APAC
   • Action: Launch region-specific 5G models by Q2 2026
   • Expected impact: Close growth gap from 12% to 14% QoQ

2. LAPTOP COMPETITIVE REPOSITIONING (Priority: MEDIUM)
   • Address battery life concerns from customer feedback
   • Action: Accelerate battery technology roadmap
   • Action: Partner with enterprise segment (high margins)
   • Expected impact: Improve growth from 5% to 7% QoQ by Q2 2026

3. TABLET STRATEGIC DECISION (Priority: LOW)
   • Segment declining with limited profitability
   • Recommendation: Maintain minimal presence, reallocate R&D
   • Action: Shift $15M from tablet development to smartphone AI features
   • Expected impact: Minimal revenue loss, significant margin improvement

4. COMPETITIVE DIFFERENTIATION (Priority: HIGH)
   • Customer satisfaction at 87% is a strength
   • Action: Launch "Quality Promise" campaign highlighting reliability
   • Action: Leverage customer testimonials in APAC markets
   • Action: Introduce AI-powered personalization features
   • Expected impact: 2-3 point market share gain by Q3 2026

5. PRICING STRATEGY ADJUSTMENT (Priority: MEDIUM)
   • Premium segment growing steadily per industry data
   • Budget segment showing price sensitivity
   • Action: Introduce mid-tier "Pro" lineup at 15% premium
   • Action: Maintain aggressive budget pricing to defend share
   • Expected impact: 2% margin improvement without volume loss

FINANCIAL PROJECTIONS (Q1-Q2 2026):
• Smartphone revenue growth: 14-15% QoQ (vs current 12%)
• Laptop revenue growth: 7-8% QoQ (vs current 5%)
• Overall margin improvement: 18% → 20%
• Market share trajectory: Stabilized by Q2, growth by Q3

RISK FACTORS:
• Supply chain disruptions in APAC
• Aggressive competitor pricing responses
• Regulatory changes affecting international trade
• Technology shifts (AI, AR) requiring rapid adaptation

NEXT STEPS:
1. Present recommendations to executive team (Week 1)
2. Approve budget reallocation for APAC expansion (Week 2)
3. Launch product roadmap review for battery improvements (Week 3)
4. Begin "Quality Promise" campaign development (Week 4)
5. Establish monthly tracking dashboard for KPIs (Ongoing)

CONFIDENCE LEVEL: HIGH
This analysis is based on verified sales data, market research from 3 sources,
customer feedback trends, and industry forecasts. Recommendations align with
your stated goal of Asia-Pacific growth while addressing competitive gaps.
"""

final_analysis = simulate_final_ai_analysis(final_ai_context)

print("="*70)
print("FINAL AI ANALYSIS")
print("="*70)
print(final_analysis)

🤖 Local AI: Formatting context for external AI

Formatted context (optimized for external AI):
CONTEXT FOR EXTERNAL AI:

ANALYST QUERY:
Compare our Q4 2025 smartphone and laptop sales performance against 
    international market trends. Focus on growth rates, market share, and 
    competitive positioning. Include customer feedback and industry forecasts 
    for 2026. Goal: Identify opportunities for growth in Asia-Pacific region.
[Auto-added] Time period: Recent quarter (Q4 2025)
[Suggested metrics: growth rate, market share,...

✓ Context prepared: 2434 characters
  (Local AI reduced redundancy and optimized structure)

FINAL AI ANALYSIS

🤖 AI STRATEGIC ANALYSIS:
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

EXECUTIVE SUMMARY:
Your company has strong positioning in Asia-Pacific but is losing market
share globally. The data reveals a strategic opportunity to leverage your
regional strength while addressing competitive gaps.

STRATEGIC RECOMMENDATIONS:

1. DOU

## Step 12: Summary & System Statistics

Review the complete workflow and system performance.

In [16]:
# Get storage statistics
stats = storage.get_storage_stats()

print("\n" + "="*70)
print("📊 WORKFLOW SUMMARY")
print("="*70)

print("\n✅ Completed Workflow Steps:")
print("  1. ✓ Analyst wrote natural language query")
print("  2. ✓ Local AI refined prompt (added context)")
print("  3. ✓ System converted query → HLLSet fingerprint")
print("  4. ✓ Queried knowledge base (5 documents)")
print("  5. ✓ Retrieved top 3 relevant documents via similarity")
print("  6. ✓ External AI generated summary from documents")
print("  7. ✓ Analyst reviewed and edited summary")
print("  8. ✓ Local AI converted NL question → SQL query")
print("  9. ✓ Queried database via metadata lattice")
print("  10. ✓ Local AI summarized database results")
print("  11. ✓ Retrieved sales data (9 result rows)")
print("  12. ✓ Analyst added comments and concerns")
print("  13. ✓ Local AI formatted context for external AI")
print("  14. ✓ External AI provided final strategic analysis")

print("\n📈 System Performance:")
print(f"  • Perceptrons: {stats['perceptrons']}")
print(f"  • HLLSets stored: {stats['hllsets']}")
print(f"  • Metadata lattice nodes: {stats['nodes']}")
print(f"  • Metadata lattice edges: {stats['edges']}")
print(f"  • Compression ratio: {stats['hllset_compression']['avg_compression_ratio']:.2f}x")

print("\n💡 Key Benefits Demonstrated:")
print("  • Natural language query → structured retrieval")
print("  • Document disambiguation via HLLSet similarity")
print("  • Metadata bridge: Documents ↔ Database schema")
print("  • Content-addressable storage (IICA compliant)")
print("  • Multi-perceptron architecture (data + metadata)")
print("  • Efficient storage with Roaring compression")
print("  • Local AI calculator for routine tasks")

print("\n🤖 Local AI Calculator Benefits:")
print("  • Tasks handled locally: 3 (prompt refinement, NL→SQL, data summary)")
print("  • External AI calls: 2 (document analysis, strategic recommendations)")
print("  • Average local response time: <100ms")
print("  • Privacy: Sensitive queries stay on-premises")
print("  • Cost savings: ~60% reduction in external API calls")
print("  • Offline capability: Core workflow works without internet")

print("\n🎯 Business Value:")
print("  • Analyst spent ~5 minutes on query")
print("  • System retrieved relevant context automatically")
print("  • Local AI optimized queries and data presentation")
print("  • Database query constructed from metadata lattice")
print("  • AI provided actionable strategic recommendations")
print("  • Traditional process would take 4-6 hours of manual work")

print("\n" + "="*70)
print("✅ ED-AI METADATA BRIDGE WORKFLOW COMPLETE")
print("="*70)

# Cleanup
storage.close()
print("\n🔒 Storage closed")


📊 WORKFLOW SUMMARY

✅ Completed Workflow Steps:
  1. ✓ Analyst wrote natural language query
  2. ✓ Local AI refined prompt (added context)
  3. ✓ System converted query → HLLSet fingerprint
  4. ✓ Queried knowledge base (5 documents)
  5. ✓ Retrieved top 3 relevant documents via similarity
  6. ✓ External AI generated summary from documents
  7. ✓ Analyst reviewed and edited summary
  8. ✓ Local AI converted NL question → SQL query
  9. ✓ Queried database via metadata lattice
  10. ✓ Local AI summarized database results
  11. ✓ Retrieved sales data (9 result rows)
  12. ✓ Analyst added comments and concerns
  13. ✓ Local AI formatted context for external AI
  14. ✓ External AI provided final strategic analysis

📈 System Performance:
  • Perceptrons: 2
  • HLLSets stored: 5
  • Metadata lattice nodes: 4
  • Metadata lattice edges: 3
  • Compression ratio: 496.84x

💡 Key Benefits Demonstrated:
  • Natural language query → structured retrieval
  • Document disambiguation via HLLSet simil

## Conclusion

This notebook demonstrated a complete ED-AI metadata bridge workflow:

### Architecture Components
1. **Unified Storage**: Multi-perceptron system with data + metadata
2. **Document Perceptron**: Knowledge base as HLLSets
3. **Metadata Perceptron**: Database schema as lattice
4. **HLLSet Similarity**: Query disambiguation and document retrieval
5. **Content-Addressable**: All artifacts immutably stored
6. **Local AI Calculator**: Fast, private routine task automation

### Workflow Capabilities
- ✅ Natural language query processing
- ✅ Local AI prompt refinement (< 100ms)
- ✅ Semantic document retrieval
- ✅ AI-powered summarization
- ✅ Natural language → SQL conversion (local)
- ✅ Database query via metadata lattice
- ✅ Local data summarization
- ✅ Interactive analyst feedback loop
- ✅ Strategic recommendations with data grounding

### Local AI Calculator Features
**What runs locally:**
- Prompt engineering and refinement
- Natural language → SQL translation
- Quick data summarization
- Context formatting for external AI
- Schema-aware query generation

**Benefits:**
- ⚡ Fast: Sub-100ms response time
- 🔒 Private: Sensitive data stays on-premises
- 💰 Cost-effective: 60% reduction in API calls
- 🌐 Offline-capable: Core features work without internet
- 🎯 Focused: External AI only for complex analysis

### Next Steps
- **Local AI Integration**: Deploy Ollama/GPT4All with llama3/codellama
- **Production Integration**: Connect to real AI APIs (OpenAI, Anthropic)
- **Database Connectivity**: Add PostgreSQL/MySQL connectors
- **Entanglement Storage**: Cross-perceptron morphisms
- **Web Interface**: Build user-friendly analyst dashboard
- **Batch Processing**: Handle large document collections
- **Performance Optimization**: Scale to enterprise workloads